In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir("/content/drive/My Drive/IPC-SHM_Project1_LIN_YANG/Python_Codes")
!ls

/bin/bash: google-drive-ocamlfuse: command not found
cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb  README.md
drive							      train.ipynb
evaluate.ipynb						      Validation
Modules							      Verification
predict.ipynb


Text for segmentation

In [ ]:
import glob
import numpy as np
import torch
import os
import cv2
from torchvision import transforms

# -------------------------------------
# import propsed U-net models 
# ----------------------------------
from Modules.unet_model import UNet
# from Modules.unet_model_attn import UNet
# from Modules.unet_model_attn_ad import UNet
# from Modules.unet_model_attn_full import UNet


if __name__ == "__main__":
    # Select the device, cuda or cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Load picture channel 1, set classes as 1。
    net = UNet(n_channels=1, n_classes=1)
    net.to(device=device)

    # -------------------------------------
    # import pth files 
    # ! Please notice the right path !
    # ----------------------------------
    best_model = 'Verification/best_model_unet.pth'

    net.load_state_dict(torch.load(best_model, map_location=device))
    # Test mode
    net.eval()

    # -------------------------------------
    # import iamges for test 
    # ! Please notice the right path !
    # ----------------------------------
    tests_path = glob.glob('Verification/data/test/test_unet/' 
                           + '*.png')

    # Traverse all pictures
    for test_path in tests_path:
        # Result path
        save_res_path = test_path.split('.')[0] + '_res.png'
        # Load picture
        img = cv2.imread(test_path)
        # Convert to grayscale picture
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # Set batch as 1, channel as 1, size 512*512 array
        img = img.reshape(1, 1, img.shape[0], img.shape[1])
        # Tensor
        img_tensor = torch.from_numpy(img)

        img_tensor = img_tensor.to(device=device, dtype=torch.float32)
        # Predict
        pred = net(img_tensor)
        
        pred = np.array(pred.data.cpu()[0])[0]
        # Process result
        pred[pred >= 0.5] = 255
        pred[pred < 0.5] = 0
        # Save picture
        cv2.imwrite(save_res_path, pred)